# Risk assessment through Satellite Scanning

This script uses `VIRASS` to generate a tree inventory along the power line. 

In [1]:
import VIRASS as ges
import geopandas
import matplotlib.pyplot as plt

c:\Users\miche\anaconda3\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
#--Load power line
power_line = ges.io.load_infrastructure_line("_data/power_lines/Kraftnett_files/power_lines.gpkg", crs = 32632)
power_line = ges.infrastructure_utils.clean_line(power_line)

# Export it
#power_line.to_file("_data/power_lines/Kraftnett_files/power_lines.gpkg", driver = "GPKG")

In [ ]:
# Create corridor
corridor = ges.infrastructure_utils.create_corridor(power_line, corridor_size = 30)

### Tree inventory generation

In [ ]:
# Tree inventory generation
tree_inventory = ges.io.load_tree_inventory("_data/trees/crowns/tree_inventory.gpkg")

if tree_inventory is None:
    # generate it
    tree_inventory = ges.infrastructure_utils.generate_tree_inventory_along_power_lines(power_line, 
                                satellite_map_file = "_data/WorldView_area.tif", 
                                tree_mask_file = "_data/trees/tree_segmentation/tree_pred_bn_Unet_attention_segmenter_1_4.tif",
                                large_corridor_side_size = 30, small_corridor_side_size = 10,
                                tree_species_map_file = "_data/trees/tree_species/R_refined_2_Q.tif", 
                                nDSM_map_file = "_data/trees/3D_modeling/nDSM_pred.tif", 
                                mode = "multiscale")        
    tree_inventory.to_file("_data/trees/crowns/tree_inventory.gpkg", driver = "GPKG") 


The tree inventory shows the location ,species, height and other parameters of all the trees in the `large_corridor_side_size` (set by default to 30 meters).
Some parameters (like DBH and shielding factors) are calculated only for trees inside `small_corridor_side_size` (set by default to 10 meters), otherwise they are NaN.

In [5]:
tree_inventory.head(5) 

,pointX,pointY,crown_radius,power_line_segment,tree_species,tree_height,dst_to_line,DBH,f_shield_N,f_shield_NE,f_shield_E,f_shield_SE,f_shield_S,f_shield_SW,f_shield_W,f_shield_NW,geometry
0,294644.41,6806476.0,1.229623,0,deciduous,1.740060,29.348850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((294645.640 6806476.000, 294645.634 6..."
1,294648.91,6806475.0,1.465808,0,deciduous,2.195107,29.295565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((294650.376 6806475.000, 294650.369 6..."
2,294650.91,6806471.5,2.092071,0,deciduous,5.798230,26.281626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((294653.002 6806471.500, 294652.992 6..."
3,294652.91,6806473.0,1.954410,0,deciduous,8.745656,28.160834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((294654.864 6806473.000, 294654.855 6..."
4,294655.41,6806470.5,2.676186,0,deciduous,6.583649,26.228341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((294658.086 6806470.500, 294658.073 6..."


## Static risk map

Among the trees on the corridor, assign value 1 to trees that can collide with power lines, 0 otherwise

In [ ]:
# Static risk map
static_risk_map = ges.infrastructure_utils.static_risk_map(tree_inventory, power_line_height = 10.8, margin = 2)
static_risk_map.to_file("_data/trees/crowns/static_risk.gpkg", driver = "GPKG")

## Dynamic risk map

Calculate the critical wind speed for all trees. 
Detect trees hat can fall due to input wind (can_fall = 1 or 0). 
For those that can fall, assign value 1 to trees that can collide with power lines (can_hit = 1 or 0) 

In [ ]:
# Dynamic risk map  
dynamic_risk_map = ges.infrastructure_utils.dynamic_risk_map(tree_inventory, power_line_height = 10.8, margin = 2, 
                                                         wind_direction = 'W',
                                                         wind_gust_speed = 20, #17.4
                                                         power_line = power_line)
dynamic_risk_map.to_file("_data/trees/crowns/dynamic_risk_20_W.gpkg", driver = "GPKG")

### Extra:

We can also compute points along the power line (sample by `point_distance`) and count how many dangerous trees there are nearby. Dangerous trees are defined as the trees that can hit the power line.

In [ ]:
points_along_power_line = ges.infrastructure_utils.assign_risk_to_power_line(power_line, static_risk_map, 
                                                                             point_distance = 20, trees_nearby = 10, 
                                                                             save_output = True, 
                                                                             save_path = "_data/power_lines/risk_map_points.gpkg")